In [5]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Load your dataset
# Assume 'StudentResponse' is the column with student responses in your dataset
data = pd.read_csv('data/Book1.csv')

# Create TF-IDF vectorizer and fit on existing student responses
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(data['StudentResponse'])


In [6]:
data

,Question,QuestionID,Identifier,StudentResponse,CorrectAnswer,MaxPossibleScore,StudentScore,TenantName,Provider,Rubric,Grade,Subject
0,Read the passage and answer the following ques...,8566818,2e8f1f27-cd27-4e18-aaaf-16d94e4bbae6,Anansi is very clever because in the story Ana...,<p>Anansi&nbsp; wonido foot of the mango Tree<...,2,2.0,Leona Group - Discovery,KDS 21-22,NaN,Grade 4,ELA
1,Christopher and his dad are planning a fishing...,8609328,4562d3ae-074b-47ec-a29e-8f8c4784b072,<ul><li><strong><em>First I added 12 &amp; 14....,<p>A.Christopher needs 24 worms in total</p><p...,2,2.0,Leona Group - ECUEA,KDS 21-22,2: The student demonstrates a sufficient under...,Grade 4,Mathematics
2,Answer the question below. Use the rubric in m...,8434663,ll-d898f9d5de13474ba982db57854f46b4,Because exosolar planets are to far away to se...,NaN,4,3.5,Galileo Academy,Lincoln Learning Solution,Answer the question below. Use the rubric in m...,Grades 9-12,Science
3,4\nMr. Alexander wrote the two expressions bel...,7857216,i-262004,"<p>&nbsp;</p><table><tbody><tr style=""height:1...","<p>&nbsp;</p><table><tbody><tr style=""height:1...",4,4.0,Lincoln Learning Solutions,Certica Solutions,4-point response: The response indicates advan...,Grade 5,Mathematics
4,4\nOysters and blue crabs often live in the sa...,7878129,i-341772,efgbnhjhmngbfvdewsq,efgbnhjhmngbfvdewsq,4,2.5,QA QC,Certica Solutions,4 Points\nThe response indicates advanced prof...,Grade 7,Mathematics
5,4\nResting heart rate is the number of times a...,7878139,i-323742,anansi is very clever because in the story Ana...,NaN,4,1.0,Tennessee Itembank,Certica Solutions,Scoring Rubric:\n\nSample Response:\n1. It is ...,Grade 7,Mathematics
6,Answer the question below. Use the rubric in t...,8502413,ll-0e0b078329404431a2afaa146e18b7ff,<p>They dont have healthy options </p><p>they ...,<p>they could hold a fundraiser and get money ...,4,3.5,Galileo Academy,Lincoln Learning Solution,Use details to support your answer.,Grades 6-8,Wellness and Nutrition
7,Acids and bases have very different properties...,8441230,ll-2ddacd05496f4a5c8255b08785ae3d65,"I would taste it, touch it, electrocute it, an...","I would taste it, touch it, electrocute it, an...",4,4.0,Galileo Academy,Lincoln Learning Solution,Answer the question below. Use the rubric in t...,Grades 9-12,Science
8,There are six basic components of nutrition th...,8502972,ll-e7e7fe50b64d4786949372f983874a4d,I need help with this one if you can help me i...,The six basic components of nutrition are wate...,4,0.0,Galileo Academy,Lincoln Learning Solution,Answer the question below. Use the rubric in t...,Grades 6-8,Wellness and Nutrition
9,Define health in your own words. What is the d...,8501102,ll-b61787ad503f4ae3a68e2d80cdbc6441,Health is like the word for being fine or how ...,Wellness is state of when you have taken actio...,4,2.0,Galileo Academy,Lincoln Learning Solution,Answer the question below. Use the rubric in t...,Grades 6-8,Wellness and Nutrition


In [12]:
def find_similar_responses_and_grade(new_response, question_id, data, tfidf_vectorizer):
    # Filter the dataset to only include responses with the same QuestionID
    filtered_data = data[data['QuestionID'] == question_id].copy()  # Use .copy() to avoid the warning

    if filtered_data.empty:
        return "No matching QuestionID found in the dataset."

    # Fit the TF-IDF vectorizer on the filtered student responses
    tfidf_matrix = tfidf_vectorizer.fit_transform(filtered_data['StudentResponse'])

    # Transform the new response using the same TF-IDF vectorizer
    new_response_vector = tfidf_vectorizer.transform([new_response])

    # Compute cosine similarity between the new response and all filtered responses
    cosine_similarities = cosine_similarity(new_response_vector, tfidf_matrix).flatten()

    # Add the similarity scores to the filtered dataset using .loc
    filtered_data.loc[:, 'CosineSimilarity'] = cosine_similarities

    # Sort the filtered data by similarity to find the most similar responses
    similar_responses = filtered_data.sort_values(by='CosineSimilarity', ascending=False)

    # Check if the highest similarity is 100%
    if similar_responses.iloc[0]['CosineSimilarity'] == 1.0:
        assigned_grade = similar_responses.iloc[0]['StudentScore']  # Assume 'Grade' is the column for grades
        return f"The new response is identical to an existing one. Assigned grade: {assigned_grade}"
    
    # Otherwise, return the top 5 most similar responses
    return similar_responses.head(5)

# Example usage:
new_response = "<ol><li>They used telescopes to see the stars and planets.</li><li>Astronomers looked at paths the planets followed in the sky.</li><li>They send spacecrafts to visit planets.</li><li>They used rovers to move on the planet.</li></ol>"
question_id = 8562580  # Replace with the actual question ID
top_similar_responses_or_grade = find_similar_responses_and_grade(new_response, question_id, data, tfidf_vectorizer)


RuntimeError: Working outside of application context.

This typically means that you attempted to use functionality that needed
the current application. To solve this, set up an application context
with app.app_context(). See the documentation for more information.

In [9]:

# Output the result
if isinstance(top_similar_responses_or_grade, str):
    print(top_similar_responses_or_grade)
else:
    print(top_similar_responses_or_grade[['StudentResponse', 'CosineSimilarity', 'StudentScore']])


The new response is identical to an existing one. Assigned grade: 4.0
